In [1]:
#!/usr/bin/env python
# coding: utf-8


import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D=True)

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import StratifiedKFold
import math

from pandarallel import pandarallel
pandarallel.initialize()

import sys
sys.path.append('/home/ss2686/03_DICTrank')
import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic


# Path where your data is stored
data_path = '../data/processed_binarised__splits/'

results = {}

def get_Mordred_columns_to_use():
    
        datasets = {}
        directory='../data/processed/'
        # Load datasets from given directory
        for foldername in os.listdir(directory):

            if not foldername.startswith('.'):  # Ignore folders starting with a dot

                print(foldername)
                file_path = os.path.join(directory, foldername, f"{foldername}_processed.csv.gz")

                if os.path.exists(file_path):
                    datasets[foldername] = pd.read_csv(file_path, compression='gzip')
                else:
                    print(f"No matching file found for folder: {foldername}")
        
        smiles_list = []
        
        for featuresets in ["sider", "DICTrank"]: 
    
            smiles_list.extend(datasets[featuresets].Standardized_SMILES.to_list())
            print(len(smiles_list))
        smiles_list = list(set(smiles_list))
        
        print(len(smiles_list))
        data = pd.DataFrame(smiles_list, columns=["Standardized_SMILES"])
        
        Ser_Mol_train = data['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_data = calc.pandas(Ser_Mol_train)
        Mordred_table_data = Mordred_table_data.astype('float')
        Mordred_table_data = Mordred_table_data.dropna(axis='columns')
        data_columns = Mordred_table_data.columns
        
        return(data_columns)
    
data_columns = get_Mordred_columns_to_use()


# Assuming PK dataset is regression and others are classification
for dataset in os.listdir(data_path):
    
    # Exclude hidden files or directories like .ipynb_checkpoints
    if dataset.startswith('.'):
        continue
    print(dataset)

    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "")  for f in all_files if not f.startswith(".ipynb_checkpoints")]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')
        test_df = pd.read_csv(test_path, compression='gzip')
        
        Ser_Mol_train = train_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_train = calc.pandas(Ser_Mol_train)
        Mordred_table_train = Mordred_table_train.astype('float')
               
        Ser_Mol_test = test_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_test = calc.pandas(Ser_Mol_test)
        Mordred_table_test = Mordred_table_test.astype('float')

        # Retain only those columns in the test dataset
        Mordred_table_train = Mordred_table_train[data_columns]
        Mordred_table_test = Mordred_table_test[data_columns]

        X_train = np.array(Mordred_table_train)
        X_test = np.array(Mordred_table_test)
        y_train = train_df[activity]
        y_test = test_df[activity]

      
        # Classification
        model = RandomForestClassifier(n_jobs=40)
            
        # Hyperparameter Optimization
        param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
            
        classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=inner_cv,
                random_state=42,
                verbose=1,
                n_jobs=40)
            
        classification_search.fit(X_train, y_train)
        best_model = classification_search.best_estimator_
            
        # Random Over-sampling 
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
        pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
        pipeline.fit(X_train, y_train)
            
        # Predict using threshold-optimized model
        predictions_train = pipeline.predict(X_train)
        probs_train = pipeline.predict_proba(X_train)[:, 1]
        probs_test = pipeline.predict_proba(X_test)[:, 1]
            
        # Use the optimize_threshold_j_statistic function to find the best threshold
        best_threshold = optimize_threshold_j_statistic(y_train, probs_train)
        #Apply the best threshold to get binary predictions on the test data
        predictions_test = (probs_test >= best_threshold).astype(int)
            
        # Calculate CV AUC using threshold-optimized model
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1, scoring='roc_auc')

        results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
            
        
            
    # Save results at each step
    pd.DataFrame(results).T.to_csv('./physicochemical_model_results.csv')
              

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./physicochemical_model_results.csv', index=False)


INFO: Pandarallel will run on 76 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
GeneExpressionSMILES
sider
MOA
CellPaintingSMILES
Cmax
DICTrank
1378
2613
1579


  2%|▊                                        | 30/1579 [00:00<00:28, 54.43it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|██▌                                    | 104/1579 [00:01<00:13, 107.10it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

                                                                                
 95%|██████████████████████████████████████▉  | 884/930 [00:29<00:03, 11.88it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:29<?, ?it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:29<?, ?it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:29<?, ?it/s]
                                                                                
100%|█████████████████████████████████████████| 930/930 [00:29<00:00, 31.76it/s]


/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return uf


  1%|▍                                           | 1/90 [00:00<00:24,  3.63it/s]
                                                                                
100%|██████████████████████████████████████████| 90/90 [00:00<00:00, 109.51it/s]


/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 930
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 46
n_resources: 20
Fitting 5 folds for each of 46 candidates, totalling 230 fits
----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities: 100%|██████████████████████| 1/1 [00:43<00:00, 43.33s/it]
